## Experimental Workflows


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils

microscope, settings = utils.setup_session()

### Automated Quality Control

In [ ]:

# do contrast test polishing
protocol = {
    "milling": {
        "polish": {
            "stages": [{
        "application_file": "autolamella",
        "cross_section": "CleaningCrossSection",
        "hfw": 40e-6,
        "height": 6.0e-07,
        "width": 6.0e-06,
        "depth": 4.0e-07,
        "milling_current": 6.0e-11,
        "milling_voltage": 3.0e3,
        "type": "Rectangle",
        "name": "AdaptivePolishing",
            }
        ],
        "point": {
            "x": 0.0,
            "y": 5e-6,}
        }
    },
    "options": {
        "experimental": {
            "adaptive_polishing": {
                "threshold": 100,
                "step_size": 5e-6,
                "step_limit": 10,
                "image_resolution": [3072, 2188],
                "image_line_integration": 20,
                "image_dwell_time": 100e-9,
            }

    }
}
}

from autolamella.workflows.experimental import adaptive_mill_polishing

adaptive_mill_polishing(microscope, settings, protocol)

## State Management Refactor

In [ ]:
%load_ext autoreload
%autoreload 2

from pprint import pprint

from autolamella.structures import AutoLamellaProtocol, AutoLamellaStage, Experiment

EXP_PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24/experiment.yaml"
PROTOCOL_PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24/protocol.yaml"
exp = Experiment.load(EXP_PATH)
pos = exp.positions[0]
protocol = AutoLamellaProtocol.load(PROTOCOL_PATH)

print(protocol.method.workflow)

print(f"Last Completed: {pos.last_completed}")
print(f"Next: ", protocol.method.get_next(pos.workflow))
print(f"Previous: ", protocol.method.get_previous(pos.workflow))
print(f"Workflow: ", protocol.method.workflow)

## AutoLamellaProtocol Class

protocol:
- name
- method:
- configuration:
- options
- supervision
- milling

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os
from pprint import pprint

from autolamella.structures import AutoLamellaProtocol, AutoLamellaStage, Experiment

BASE_PATH = "/home/patrick/github/autolamella/autolamella/protocol/protocol-*-new.yaml"

# TODO: TRENCH MILLING METHOD

filenames = glob.glob(BASE_PATH)
pprint(filenames)
 

PROTOCOL_PATH =  "/home/patrick/github/autolamella/autolamella/protocol/protocol-waffle-new.yaml"

protocol = AutoLamellaProtocol.load(PROTOCOL_PATH)

# pprint(protocol.to_dict()["name"])

# pprint()
# from fibsem.milling import 
# protocol.milling

pprint(protocol.tmp)
# pprint(protocol.options)
# print(protocol.supervision)
# print(protocol.method.workflow)

# from fibsem.structures import FibsemImage
# from fibsem.milling.patterning.plotting import draw_milling_patterns

# stages = protocol.milling["undercut"]

# image = FibsemImage.generate_blank_image(hfw=stages[0].milling.hfw)
# draw_milling_patterns(image, stages)


pprint(protocol.to_dict())

In [ ]:
from fibsem import utils

from autolamella.protocol.validation import validate_protocol

protocol = validate_protocol(utils.load_protocol(protocol_path=PROTOCOL_PATH))

protocol2 = AutoLamellaProtocol.from_dict(protocol)
pprint(protocol2.to_dict())

In [ ]:
from autolamella.structures import AutoLamellaMethod

[m.name for m in AutoLamellaMethod]

## Experiment Review Tools

- Time estimatation for remaining
- Display Images for each workflow stage
- Generate a Report (pdf)

In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os
from pprint import pprint
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from fibsem.structures import FibsemImage

from autolamella.structures import (
    AutoLamellaMethod,
    AutoLamellaProtocol,
    AutoLamellaStage,
    Experiment,
    Lamella,
    get_completed_stages,
)

EXP_PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24/experiment.yaml"
PROTOCOL_PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24/protocol.yaml"

PATH = "/home/patrick/data/monash-cryo-em/AutoLamella-Exports" 
filenames = glob.glob(os.path.join(PATH, "**/experiment.yaml"), recursive=True)
filenames.insert(0, EXP_PATH)


# pprint(filenames)
# EXP_PATH = filenames[0]


exp = Experiment.load(EXP_PATH)
pos = exp.positions[0]
protocol = AutoLamellaProtocol.load(PROTOCOL_PATH)

# print(protocol.method.workflow)



In [ ]:

def get_workflow_snapshot(pos: Lamella, wf: AutoLamellaStage, target_size: int = 256) -> np.ndarray:
    """Get a snapshot of a workflow stage for a given position"""

    if not isinstance(target_size, int):
        target_size = int(target_size)

    # get the final high res images
    filenames = glob.glob(os.path.join(pos.path, f"*{wf.name}*final_high_res*.tif*"))
    
    if len(filenames) == 0:
        print(f"No images found for {pos.name} - {wf.name}")
        return None

    # resize and stack the images for display
    sarr = None
    for fname in sorted(filenames):
        img = FibsemImage.load(fname)
        shape = img.data.shape
        resize_shape = (int(shape[0] * (target_size / shape[1])), target_size)
        arr = np.asarray(PIL.Image.fromarray(img.data).resize(resize_shape[::-1]))
        
        # stack
        if sarr is None:
            sarr = arr
        else:
            sarr = np.append(sarr, arr, axis=1)
    return sarr

def convert_figure_to_np_array(figure: plt.Figure) -> np.ndarray:
    """Convert a matplotlib figure to a numpy array"""
    # Draw the figure
    figure.canvas.draw()
    # Get the RGBA buffer
    buf = figure.canvas.buffer_rgba()
    # Convert to numpy array
    arr = np.asarray(buf)
    plt.close()
    return arr

target_size = 1536 / 4

for filename in filenames[1:2]:

    print(f"Experiment: {os.path.basename(filename)}")
    exp = Experiment.load(filename)

    for i, pos in enumerate(exp.positions):
        print("-"*80)
        print(pos.name, pos.path)

        stages_completed = get_completed_stages(pos, method=protocol.method)

        for wf in stages_completed:

            # if not on same computer
            if not os.path.exists(pos.path):
                print(f"Path does not exist: {pos.path}, remapping to experiment path")
                pos.path = os.path.join(exp.path, pos.name)

            snapshot = get_workflow_snapshot(pos, wf, target_size=target_size)

            if snapshot is None:
                print(f"Skipping {pos.name} - {wf.name}, no images found.")
                continue

            cx1 = snapshot.shape[1] // 4
            cx2 = snapshot.shape[1] // 2 + cx1
            cy = snapshot.shape[0] // 2

            fig = plt.figure()
            plt.title(f"Lamella {pos.name} - {pos.states[wf].completed}")
            plt.imshow(snapshot, cmap="gray")
            plt.plot([cx1, cx2], [cy, cy], "y+", ms=20)
            plt.axis("off")
            plt.show()

            array = convert_figure_to_np_array(fig)
            print(array.shape)

            # save array using PIL
            # img = PIL.Image.fromarray(array)
            # img.save(f"lamella_{pos.name}_{wf.name}.png")

            # break

            # TODO: scalebar


### Review - Report Gen


In [ ]:
%load_ext autoreload
%autoreload 2

import glob
import os
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
from fibsem.structures import FibsemImage

from autolamella.structures import (
    AutoLamellaMethod,
    AutoLamellaProtocol,
    AutoLamellaStage,
    Experiment,
    Lamella,
    get_completed_stages,
)
from autolamella.tools.data import calculate_statistics_dataframe
from autolamella.tools.reporting import generate_report

EXP_PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24/experiment.yaml"  # v.0.4.0
# PROTOCOL_PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24/protocol.yaml"

PATH = "/home/patrick/data/monash-cryo-em/AutoLamella-Exports" # 0.3.4+
filenames = glob.glob(os.path.join(PATH, "**/experiment.yaml"), recursive=True)
BBQ_PATH = "/home/patrick/data/monash-cryo-em/AutoLamella-ZebraFish-Waffle-2024-07-17-11-05/experiment.yaml"
filenames.insert(0, EXP_PATH)
filenames.append(BBQ_PATH)

pprint(filenames)

In [ ]:
dfs = calculate_statistics_dataframe(os.path.dirname(filenames[2]), encoding="cp1252")

df_experiment, df_history, df_beam_shift, df_steps, df_stage, df_det, df_click, df_milling =  dfs

display(df_history)
display(df_steps)


In [ ]:
generate_report(filename=EXP_PATH, output_filename="Autolamella-export.pdf", encoding="utf-8")

In [ ]:
for filename in filenames:
    print(f"Generating report for: {filename}")

    output_filename = os.path.basename(os.path.dirname(filename)) + ".pdf"

    encoding = "cp1252"
    if "2025-01-10" in filename:
        continue

    generate_report(filename=filename, output_filename=output_filename, encoding=encoding)

In [ ]:
import logging
import os

from autolamella.structures import AutoLamellaMethod, AutoLamellaStage
from autolamella.tools.reporting import plot_lamella_summary, generate_duration_data
from autolamella.tools.reporting import plot_lamella_milling_workflow
from copy import deepcopy
import pandas as pd

PATH = "/home/patrick/data/monash-cryo-em/AutoLamella-Exports/autolamella/20241022_Sai/AutoLamella-2024-10-22-09-37"
PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24"
PATH = "/home/patrick/data/monash-cryo-em/AutoLamella-ZebraFish-Waffle-2024-07-17-11-05"

logging.basicConfig(level=logging.INFO)


for filename in filenames[2:3]:
    exp = Experiment.load(filename)
    print(f"Experiment: {exp.name}")
    print(f"Total Lamella: {len(exp.positions)}, Finished Lamella: {len(exp.at_stage(AutoLamellaStage.Finished))})")
    print(f"Failed Lamella: {[l.name for l in exp.at_failure()]}")

    for p in exp.positions:
        print(p.name)

        p.path = os.path.join(exp.path, p.name) # for remapping on different machine
        
        method = AutoLamellaMethod.ON_GRID
        if "Waffle" in exp.name:
            method = AutoLamellaMethod.WAFFLE

    #     # # TODO: resize to 256x256?
    #     # fig = plot_lamella_summary(p, method=method)
    #     # plt.show()
    #     # plt.close()
    
        fig = plot_lamella_milling_workflow(p)
        plt.show()
    df = exp.to_protocol_dataframe()
    display(df)

# save as milling_protocol.csv
df.to_csv("milling_protocol.csv", index=False)

### Experiment - Time Estimation


In [ ]:
%load_ext autoreload
%autoreload 2

import logging
logging.basicConfig(level=logging.INFO)
from typing import Dict
from fibsem.utils import format_duration
from autolamella.structures import Experiment, AutoLamellaMethod, AutoLamellaProtocol
from fibsem.milling import get_milling_stages, FibsemMillingStage, plot_milling_patterns, estimate_total_milling_time
from pprint import pprint
from fibsem.structures import FibsemImage

PATH = "/home/patrick/github/autolamella/autolamella/log/AutoLamella-2025-01-10-14-24/experiment.yaml"

exp = Experiment.load(PATH) # TODO: exp.method is incorrect
protocol = AutoLamellaProtocol.load(PATH.replace("experiment.yaml", "protocol.yaml"))
print(protocol.method)

exp.method = AutoLamellaMethod.WAFFLE
estimated_remaining_time = exp.estimate_remaining_time()

print(f"Experiment: {exp.name}: {format_duration(estimated_remaining_time)} remaining time...")

import datetime 
# get finish time based on current time
finish_time = datetime.datetime.now() + datetime.timedelta(seconds=estimated_remaining_time)
print(f"Finish time: {finish_time.strftime('%Y-%m-%d %H:%M:%S')}")


### Infinite Minimap


In [65]:
%load_ext autoreload
%autoreload 2

from fibsem.structures import FibsemImage, FibsemImageMetadata
import os, glob
import numpy as np
import matplotlib.pyplot as plt

# PATH = "/home/patrick/github/fibsem/fibsem/log/overview-image-2025-02-26_10-34-53"
PATH = "/home/patrick/github/fibsem/fibsem/log/"



import napari
from fibsem.imaging.tiled import calculate_reprojected_stage_position
from fibsem.structures import FibsemStagePosition, Point, BeamType
from copy import deepcopy

viewer = napari.Viewer()

filenames = sorted(glob.glob(os.path.join(PATH, "*overview-image-2025-02-26_10-34-53*.tif")))
print(len(filenames), filenames)


colors = ["gray", "green", "red", "blue", "orange", "purple", "pink", "cyan"]



viewer.scale_bar.visible = True
viewer.scale_bar.unit = "m"
viewer.axes.visible = True
viewer.axes.colored = False
viewer.dims.axis_labels = ["z", "y", "x"]

# rotation matrix around x-axis
theta = np.deg2rad(35)

from typing import Tuple

def get_transform(r: float) -> Tuple[np.ndarray, np.ndarray]:
    m = np.array([[1, 0, 0],
    [0, np.cos(r), -np.sin(r)],
    [0, np.sin(r), np.cos(r)]])
    minv = np.linalg.inv(m)
    return m, minv

# notes: depends on orientation, scan rotation too
def to_sample_stage(pos: FibsemStagePosition, r: float) -> FibsemStagePosition:
    m, minv = get_transform(r)

    p = np.array([pos.x, pos.y, pos.z])
    q = np.dot(m, p)
    
    return FibsemStagePosition(x=q[0], y=q[1], z=q[2], r=pos.r, t=pos.t, coordinate_system="sample-stage")

def from_sample_stage(pos: FibsemStagePosition, r: float) -> FibsemStagePosition:
    m, minv = get_transform(r)
    p = np.array([pos.x, pos.y, pos.z])
    q = np.dot(minv, p)
    
    return FibsemStagePosition(x=q[0], y=q[1], z=q[2], r=pos.r, t=pos.t, coordinate_system="RAW")


def to_napari_pos(image: np.ndarray, pos: FibsemStagePosition, pixelsize: float) -> Point:
    """Convert a sample-stage coordinate to a napari image coordinate"""
    pos = Point(
        x = pos.x - image.shape[1] * pixelsize / 2,
                y = -pos.y - image.shape[0] * pixelsize / 2)
    return pos

def from_napari_pos(image: np.ndarray, pos: Point, pixelsize: float) -> FibsemStagePosition:
    """Convert a napari image coordinate to a sample-stage coordinate"""
    p = FibsemStagePosition(
        x = pos.x + image.shape[1] * pixelsize / 2,
        y = -pos.y - image.shape[0] * pixelsize / 2)
    
    # r = np.array([
    #     [1, 0], [0, -1]
    # ])
    # t = np.array([
    #     [image.shape[1] * pixelsize / 2,
    #     -image.shape[0] * pixelsize / 2]
    # ]).T
    # print(r)
    # print(t)
    # p2 = np.dot(r, np.array([pos.x, pos.y])) + t
    # print(p)
    # print(p2)
    # TODO: we need to fill in the zrt values?

    return p

def get_stage_orientation():
    pass


def on_click(layer, event):
    print('-'*80)
    pos = event.position # world coordinate
    print(f"Event callback: {layer.name}, x={pos[1]*1e6:.2f}, y={pos[0]*1e6:.2f}")

    # note: we need to get the sample-stage z from somewhere? -> stage
    z = 0 # tmp
    
    ssp = FibsemStagePosition(x=pos[1], y=pos[0], z=z, r=None, t=None, coordinate_system="sample-stage")
    print(f"Sample Stage Position: ")
    print(ssp.to_dict())
    print(layer.metadata["sample_stage_position"])
    
    ss = from_sample_stage(ssp, np.deg2rad(pretilt))
    print("Stage Position")
    print(ss.to_dict())
    print(layer.metadata["stage_position"])


for i, fname in enumerate(filenames):
    image = FibsemImage.load(fname)
    pixelsize = image.metadata.pixel_size.x
    stage_position = image.metadata.microscope_state.stage_position
    pretilt = image.metadata.system.stage.shuttle_pre_tilt
    rot_ref = image.metadata.system.stage.rotation_reference
    rot_180 = image.metadata.system.stage.rotation_180

    beam_type = image.metadata.image_settings.beam_type
    if beam_type is BeamType.ELECTRON:
        scan_rotation = image.metadata.microscope_state.electron_beam.scan_rotation
    if beam_type is BeamType.ION:
        scan_rotation = image.metadata.microscope_state.ion_beam.scan_rotation
    # from pos, pretilt, rot_ref and rot_180 we can calculate orientation

    # scan rotation, rotates the sample stage around the z-axis (invert x, y)

    ssp = to_sample_stage(stage_position, np.deg2rad(pretilt))
    cmap = colors[i % len(colors)]

    ss = from_sample_stage(ssp, np.deg2rad(pretilt))

    # napari coordinate is y down, x right, based at top left
    # need to offset ssp, by half image size, and invert y
    pos = to_napari_pos(image.data, ssp, pixelsize)
    pos2 = from_napari_pos(image.data, pos, pixelsize)
    assert np.isclose(ssp.x, pos2.x)
    assert np.isclose(ssp.y, pos2.y)

    layer= viewer.add_image(image.data, 
                     name=os.path.basename(fname), 
                     scale=[pixelsize, pixelsize], 
                     translate=[pos.y, pos.x],
                     colormap=cmap,
                     metadata={"stage_position": stage_position.to_dict(), 
                               "sample_stage_position": deepcopy(ssp.to_dict())})
    layer.mouse_drag_callbacks.append(on_click)

napari.run()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2 ['/home/patrick/github/fibsem/fibsem/log/overview-image-2025-02-26_10-34-53-autogamma.tif', '/home/patrick/github/fibsem/fibsem/log/overview-image-2025-02-26_10-34-53.tif']


In [ ]:
{"affine": [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]], "rotate": [[1.0, 0.0], [0.0, 1.0]], "scale": [4.8828125e-07, 4.8828125e-07], "shear": [0.0], "translate": [0.0005, -0.0005]}